# Interfaces con otros lenguajes: Fortran

## Ejemplo 1: Rotación de vectores

Supongamos que queremos resolver el problema de la rotación de vectores en el espacio usando los [tres ángulos de Euler](https://mathworld.wolfram.com/EulerAngles.html). 

Dado un array con los tres ángulos de Euler, calculamos la matriz de rotación (de 3 x 3), y la aplicamos (esto es, multiplicamos) a un arreglo de _N_ vectores.



En una primer versión, podemos hacerlo en Python directamente en el notebook:

In [ ]:
import numpy as np

def matrix_rotation(angles):
  cx, cy, cz = np.cos(angles)
  sx, sy, sz = np.sin(angles)
  R = np.zeros((3, 3))
  R[0, 0] = cx * cz - sx * cy * sz
  R[0, 1] = cx * sz + sx * cy * cz
  R[0, 2] = sx * sy

  R[1, 0] = -sx * cz - cx * cy * sz
  R[1, 1] = -sx * sz + cx * cy * cz
  R[1, 2] = cx * sy

  R[2, 0] = sy * sz
  R[2, 1] = -sy * cz
  R[2, 2] = cy
  return R


def rotate(angles, v):
  return np.dot(matrix_rotation(angles), v)

In [ ]:
N = 100
# Ángulos de Euler
angle = np.random.random(3)
# Definimos N vectores tridimensionales
v = np.random.random((3, N))

Si ya tenemos un módulo donde están programadas las funciones necesarias 

In [ ]:
# %load rotacion_p.py
#! /usr/bin/ipython3
import numpy as np


def matrix_rotation(angles):
  cx, cy, cz = np.cos(angles)
  sx, sy, sz = np.sin(angles)
  R = np.zeros((3, 3))
  R[0, 0] = cx * cz - sx * cy * sz
  R[0, 1] = cx * sz + sx * cy * cz
  R[0, 2] = sx * sy

  R[1, 0] = -sx * cz - cx * cy * sz
  R[1, 1] = -sx * sz + cx * cy * cz
  R[1, 2] = cx * sy

  R[2, 0] = sy * sz
  R[2, 1] = -sy * cz
  R[2, 2] = cy
  return R


def rotate(angles, v):
  return np.dot(matrix_rotation(angles), v)


es fácil utilizarlas:

In [ ]:
N = 100
# Ángulos de Euler
angle = np.random.random(3)
# Definimos N vectores tridimensionales
v = np.random.random((3, N))

In [ ]:
y = rotate(angle,v)

In [ ]:
print(angle)
print(y[:,0:5].T)

### Importando desde un módulo en un archivo

También podemos tener las funciones para la rotación en un módulo `rotaciones.py`. Recordemos cómo se importarían en este caso

In [ ]:
pwd

In [ ]:
cd ../scripts/interfacing_python

In [ ]:
ls

In [ ]:
import rotaciones as rotp

In [ ]:
help(rotp)

In [ ]:
yp = rotp.rotate(angle,v)

In [ ]:
print(yp[:,0:5].T)
np.allclose(y,yp)

## Interfaces con Fortran

Veamos cómo trabajar si tenemos el código para realizar las rotaciones en Fortran

### Primer ejemplo: Nuestro código

El código en Fortran que tenemos es:

```fortran
function rotate(theta, v, N) result(y)
  implicit none
  integer :: N
  real(8), dimension(3), intent(IN) :: theta
  real(8), dimension(3,N), intent(IN) :: v
  real(8), dimension(3,N) :: y
  real(8), dimension(3,3) :: R
  real(8) :: cx, cy, cz, sx, sy, sz

  ! Senos y Cosenos de los tres ángulos de Euler 
  cx = cos(theta(1)); cy = cos(theta(2)); cz = cos(theta(3))
  sx = sin(theta(1)); sy = sin(theta(2)); sz = sin(theta(3))

  ! Matriz de rotación
  R(1,1) = cx*cz - sx*cy*sz
  R(1,2) = cx*sz + sx*cy*cz
  R(1,3) = sx*sy

  R(2,1) = -sx*cz - cx*cy*sz
  R(2,2) = -sx*sz + cx*cy*cz
  R(2,3) = cx*sy

  R(3,1) = sy*sz
  R(3,2) = -sy*cz
  R(3,3) = cy

  ! Aplicamos la rotación
  y = matmul(R, v)
end function rotate
```

In [ ]:
cd ../interfacing_F

### F2PY
F2PY  -Fortran to Python interface generator- es una utilidad que permite generar una interface para utilizar funciones y datos definidos en Fortran desde Python.

Información sobre la interfaz entre Fotran y Python, y en particular sobre F2PY, puede encontrarse en:

- [Scipy cookbook](http://scipy-cookbook.readthedocs.io/items/idx_interfacing_with_other_languages.html)
- [F2PY Users Guide and Reference Manual](https://docs.scipy.org/doc/numpy-dev/f2py/index.html)
- [Fortran Best Practices](http://www.fortran90.org/src/best-practices.html#interfacing-with-python)
- http://websrv.cs.umt.edu/isis/index.php/F2py_example

El primer paso es utilizar esta utilidad:
```bash
$ f2py3 -c rotacion.f90 -m rotacion_f
```

In [ ]:
!f2py3 -c rotacion.f90 -m rotacion_f

In [ ]:
from rotacion_f import rotaciones as rotf

In [ ]:
yf = rotf.rotate(angle, v)
print(y[:,0:5].T)
print(yf[:,0:5].T)
np.allclose(yf,y)

Veamos qué es exactamente lo que importamos:

In [ ]:
np.info(rotf.rotate)

Como vemos, estamos usando la función `rotate` definida en Fortran. Notar que:
* Tiene tres argumentos.
   * Dos argumentos requeridos: `angles` y `v`
   * Un argumento, correspondiente a la dimensión `n` que F2PY automáticamente detecta como opcional.

             

### Segundo Ejemplo: Código heredado

La conversión que realizamos con f2py3 la podríamos haber realizado en dos pasos:

```bash
$ f2py3 rotacion.f90 -m rotacion_f -h rotacion.pyf
$ f2py3 -c rotacion.pyf -m rotacion_f
```
En el primer paso se crea un archivo *signature* que después se utiliza para crear el módulo que llamaremos desde **Python**. Haciéndolo en dos pasos nos permite modificar el texto del archivo *.pyf* antes de ejecutar el segundo comando.

Esto es útil cuando el código original no es lo suficientemente "moderno", no tiene toda la información necesaria sobre los argumentos o es un código que uno no quiere o puede editar. Veamos que forma tienen con un ejemplo más simple (tomado de la [guía de usuario](https://docs.scipy.org/doc/numpy-dev/f2py/getting-started.html)):

```fortran
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
```

In [ ]:
!f2py3 --overwrite-signature fib1.f -m fib1 -h fib1.pyf

In [ ]:
!cat fib1.pyf

El contenido del archivo `fib1.pyf` es:

```fortran90
python module fib1 ! in 
    interface  ! in :fib1
        subroutine fib(a,n) ! in :fib1:fib1.f
            real*8 dimension(n) :: a
            integer, optional,check(len(a)>=n),depend(a) :: n=len(a)
        end subroutine fib
    end interface 
end python module fib1
```

Este código indica que tenemos una subrutina que toma dos argumentos:
- `a` es un array
- `n` es un entero opcional, que tiene que ser mayor que `len(a)`

In [ ]:
!f2py3  -c fib1.pyf fib1.f 

In [ ]:
ls

In [ ]:
import fib1

In [ ]:
print(fib1.fib.__doc__)

In [ ]:
a = np.zeros(12)
fib1.fib(a)
print(a)

In [ ]:
a = np.zeros(12)
fib1.fib(a,8)
print(a)

In [ ]:
a = np.zeros(12)
fib1.fib(a,18)
print(a)

Esta es una de las características de F2PY: hace un chequeo básico de los argumentos. Hay otro error que no llega a atrapar: Si le pasamos un array que no es del tipo indicado, falla (sin avisar). Éste claramente no es el comportamiento deseado:

In [ ]:
a = np.zeros(12, dtype=int)
fib1.fib(a)
print(a)

Vamos a modificar el archivo de *signature* para enseñarle dos cosas:

- El entero es un argumento de entrada (requerido)
- El *array* `a` es un archivo de salida **exclusivamente**. Entonces no debemos dárselo. La parte `dimension(n)` y `depend(n)` indica que debe crear un vector de ese tamaño.


In [ ]:
!cat fib1.pyf 

In [ ]:
!cat fib2.pyf

In [ ]:
!f2py3  -c fib2.pyf fib1.f > /dev/null

In [ ]:
import fib2
print(fib2.fib.__doc__)

In [ ]:
fib2.fib(9)

In [ ]:
print(fib2.fib(14))

La segunda manera de arreglar este problema, en lugar de modificar el archivo de *signature* podría haber sido modificar el código (o hacer una rutina intermedia). Agregando comentarios de la forma `Cf2py` no influimos en la compilación `Fortran` pero F2PY los reconoce. En este caso le damos la información sobre la intención de los argumentos en el código:

```fortran
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
Cf2py intent(in) n
Cf2py intent(out) a
Cf2py depend(n) a
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
```

In [ ]:
!f2py3  -c fib3.f -m fib3 > /dev/null

In [ ]:
import fib3
print(fib3.fib.__doc__)

In [ ]:
print(fib2.fib.__doc__)

como vemos, son exactamente iguales.

### F2PY para código en C

Es posible usar F2PY para código escrito en C, pero en ese caso debemos escribir el *signature file* a mano.

Para código en C es conveniente utilizar **Cython**. Cython es un lenguaje de programación pensado para hacer más fácil escribir extensiones a Python en C. Uno escribe el código en Cython, y luego es traducido a C, con optimizaciones.

**Cython** también puede utilizarse con Fortran de una manera similar a cómo se usa con C. Para más información ver la [documentación oficial](http://docs.cython.org/en/latest/index.html)
